<a href="https://colab.research.google.com/github/sjtometich/takehome/blob/waymark/Sara_Tometich_Waymark_Assignment_20240605.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes

- I used databricks to code out the assignment since it has some pre-installed packages for accessing data on s3

# Get Data

In [ ]:
from pyspark.sql.functions import to_date, date_format, col, last_day

s3_bucket_name = 'waymark-assignment'
file_name_1 = 'patient_id_month_year.csv'
file_name_2 = 'outpatient_visits_file.csv'
patient_id_month_year = f's3://{s3_bucket_name}/{file_name_1}'
outpatient_visits_file = f's3://{s3_bucket_name}/{file_name_2}'

# Set up AWS credentials
spark.conf.set('spark.hadoop.fs.s3.access.key', 'aws_access_key_id')
spark.conf.set('spark.hadoop.fs.s3.secret.key', 'aws_secret_access_key')

# Load enrollment data
enroll = spark.read.options(header='True').csv(patient_id_month_year)
# Fix date formatting
enroll = enroll.withColumn("enrollment_start_date", date_format(to_date(col("month_year"), "M/d/yy"), "yyyy-MM-dd"))
enroll = enroll.withColumn("enrollment_end_date", last_day(date_format(to_date(col("month_year"), "M/d/yy"), "yyyy-MM-dd")))

# Load outpatient visits
op_visits = spark.read.options(header='True').csv(outpatient_visits_file)
# Fix date formatting
op_visits = op_visits.withColumn("visit_date", date_format(to_date(col("date"), "M/d/yy"), "yyyy-MM-dd"))

# Enrollment Span

In [ ]:
# Viewing data to make sure the date transformations are behaving the way I expect

enroll.show()

+----------+----------+----+---------------------+-------------------+
|patient_id|month_year| _c2|enrollment_start_date|enrollment_end_date|
+----------+----------+----+---------------------+-------------------+
|    ID0001|    1/1/23|null|           2023-01-01|         2023-01-31|
|    ID0001|    2/1/23|null|           2023-02-01|         2023-02-28|
|    ID0001|    3/1/23|null|           2023-03-01|         2023-03-31|
|    ID0001|    6/1/23|null|           2023-06-01|         2023-06-30|
|    ID0001|    8/1/23|null|           2023-08-01|         2023-08-31|
|    ID0001|   11/1/23|null|           2023-11-01|         2023-11-30|
|    ID0001|   12/1/23|null|           2023-12-01|         2023-12-31|
|    ID0002|    2/1/23|null|           2023-02-01|         2023-02-28|
|    ID0002|    3/1/23|null|           2023-03-01|         2023-03-31|
|    ID0002|    4/1/23|null|           2023-04-01|         2023-04-30|
|    ID0002|    7/1/23|null|           2023-07-01|         2023-07-31|
|    I

### Check for Nulls/Blanks

#### Nulls

In [ ]:
from pyspark.sql.functions import col

enroll.filter(col("patient_id").isNull()).count()

Out[64]: 1039817

In [ ]:
from pyspark.sql.functions import col

enroll.filter(col("month_year").isNull()).count()

Out[65]: 1039817

In [ ]:
from pyspark.sql.functions import col

enroll.filter(col("enrollment_start_date").isNull()).count()

Out[66]: 1039817

In [ ]:
from pyspark.sql.functions import col

enroll.filter(col("enrollment_end_date").isNull()).count()

Out[67]: 1039817

#### Blanks

In [ ]:
from pyspark.sql.functions import col

enroll.filter(col("patient_id") == "").count()

Out[68]: 0

In [ ]:
from pyspark.sql.functions import col

enroll.filter(col("month_year") == "").count()

Out[69]: 0

In [ ]:
from pyspark.sql.functions import col

enroll.filter(col("enrollment_start_date") == "").count()

Out[70]: 0

In [ ]:
from pyspark.sql.functions import col

enroll.filter(col("enrollment_end_date") == "").count()

Out[71]: 0

### Total Row Count

In [ ]:
# Total row count

enroll.count()

Out[72]: 1047126

### Clean Data

- Drop nulls, no longer needed columns, and duplicates

In [ ]:
# Drop extra null columns
from pyspark.sql.functions import col

enroll_upd = enroll.drop('_c2', 'month_year')

# Remove duplicates and remove rows where patient ID is null
enroll_clean = enroll_upd.dropDuplicates().filter(col("patient_id").isNotNull())

#### Double checks after data cleaing

- Final schema contains the columns we want and in the right format

In [ ]:
enroll_clean.count()

Out[74]: 7309

In [ ]:
enroll_clean.show()

+----------+---------------------+-------------------+
|patient_id|enrollment_start_date|enrollment_end_date|
+----------+---------------------+-------------------+
|    ID0002|           2023-09-01|         2023-09-30|
|    ID0002|           2023-03-01|         2023-03-31|
|    ID0001|           2023-01-01|         2023-01-31|
|    ID0003|           2023-06-01|         2023-06-30|
|    ID0003|           2023-04-01|         2023-04-30|
|    ID0003|           2023-03-01|         2023-03-31|
|    ID0003|           2023-12-01|         2023-12-31|
|    ID0001|           2023-02-01|         2023-02-28|
|    ID0001|           2023-06-01|         2023-06-30|
|    ID0002|           2023-10-01|         2023-10-31|
|    ID0002|           2023-02-01|         2023-02-28|
|    ID0002|           2023-12-01|         2023-12-31|
|    ID0001|           2023-11-01|         2023-11-30|
|    ID0003|           2023-07-01|         2023-07-31|
|    ID0001|           2023-12-01|         2023-12-31|
|    ID000

## Write & Read Enrollment Span CSV

In [ ]:
# s3_output_path = "s3://waymark-assignment/enrollment_span.csv"
# enroll_clean.write.option("header", True).mode("overwrite").csv(s3_output_path)

local_output_path = "/Users/saratometich/Downloads/enrollment_span.csv"
enroll_clean.write.option("header", True).mode("overwrite").csv(local_output_path)

In [ ]:
enrollment_span_csv = spark.read.options(header='True').csv("/Users/saratometich/Downloads/enrollment_span.csv")

# Answer 1

Report the number of rows in patient_enrollment_span.csv

ANSWER: 7309

In [ ]:
enrollment_span_csv.count()

Out[78]: 7309

# Outpatient Visits

In [ ]:
# Viewing data to make sure the date transformations are behaving the way I expect

op_visits.show()

+----------+--------+----------------------+----+----+----+----+----+----------+
|patient_id|    date|outpatient_visit_count| _c3| _c4| _c5| _c6| _c7|visit_date|
+----------+--------+----------------------+----+----+----+----+----+----------+
|    ID0001|  1/3/23|                     2|null|null|null|null|null|2023-01-03|
|    ID0001|  1/8/23|                     2|null|null|null|null|null|2023-01-08|
|    ID0001|  1/9/23|                     2|null|null|null|null|null|2023-01-09|
|    ID0001| 1/15/23|                     2|null|null|null|null|null|2023-01-15|
|    ID0001| 1/21/23|                     4|null|null|null|null|null|2023-01-21|
|    ID0001| 1/25/23|                     1|null|null|null|null|null|2023-01-25|
|    ID0001| 3/11/23|                     3|null|null|null|null|null|2023-03-11|
|    ID0001| 3/22/23|                     1|null|null|null|null|null|2023-03-22|
|    ID0001|  6/3/23|                     2|null|null|null|null|null|2023-06-03|
|    ID0001|  6/3/23|       

### Clean Data

In [ ]:
# Drop extra null columns
from pyspark.sql.functions import col

op_visits_upd = op_visits.drop('_c3', '_c4', '_c5', '_c6', '_c7', 'date')

# Remove duplicates and remove rows where patient ID is null
op_visits_clean = op_visits_upd.dropDuplicates().filter(col("patient_id").isNotNull())

#### Double checks after data cleaing

- Final schema contains the columns we want and in the right format

In [ ]:
op_visits.count()

Out[81]: 1043905

In [ ]:
op_visits_clean.count()

Out[82]: 15196

In [ ]:
op_visits_clean.show()

+----------+----------------------+----------+
|patient_id|outpatient_visit_count|visit_date|
+----------+----------------------+----------+
|    ID0001|                     1|2023-03-22|
|    ID0001|                     1|2023-09-17|
|    ID0001|                     3|2023-06-14|
|    ID0002|                     1|2023-01-15|
|    ID0001|                     2|2023-09-06|
|    ID0001|                     4|2023-07-17|
|    ID0001|                     2|2023-11-13|
|    ID0001|                     3|2023-08-31|
|    ID0001|                     2|2023-12-15|
|    ID0001|                     4|2023-01-21|
|    ID0001|                     1|2023-01-25|
|    ID0001|                     2|2023-06-03|
|    ID0001|                     2|2023-01-03|
|    ID0001|                     2|2023-01-15|
|    ID0001|                     2|2023-01-08|
|    ID0001|                     2|2023-01-09|
|    ID0001|                     4|2023-09-06|
|    ID0001|                     1|2023-06-03|
|    ID0001| 

## ct_outpatient_visits

The number of outpatient visits a patient had within the enrollment period (between enrollment_start_date and enrollment_end_date)

### Code

In [ ]:
from pyspark.sql.functions import col, coalesce, lit, sum as pyspark_sum

joined_df = enrollment_span_csv.join(op_visits,
                                 (op_visits.patient_id == enrollment_span_csv.patient_id) &
                                 (op_visits.visit_date <= enrollment_span_csv.enrollment_end_date) &
                                 (op_visits.visit_date >= enrollment_span_csv.enrollment_start_date),
                                 "left")

grouped_df = joined_df.groupBy("enrollment_start_date", "enrollment_end_date", enrollment_span_csv["patient_id"].alias("patient_id"))

ct_outpatient_visits = grouped_df.agg(coalesce(pyspark_sum("outpatient_visit_count"), lit(0)).alias("ct_outpatient_visits"))

### Checks

In [ ]:
# Expect no records lost from enrollment span to this point

ct_outpatient_visits.count()

Out[85]: 7309

## ct_days_with_outpatient_visit

The number of distinct days within an enrollment period (between enrollment_start_date and enrollment_end_date) when the patient had one or more outpatient visit

### Code

In [ ]:
from pyspark.sql.functions import col, countDistinct, sum as pyspark_sum

joined_df = enrollment_span_csv.join(op_visits,
                                 (op_visits.patient_id == enrollment_span_csv.patient_id) &
                                 (op_visits.visit_date <= enrollment_span_csv.enrollment_end_date) &
                                 (op_visits.visit_date >= enrollment_span_csv.enrollment_start_date),
                                 "left")

grouped_df = joined_df.groupBy(enrollment_span_csv["patient_id"].alias("patient_id"), "enrollment_start_date", "enrollment_end_date")

result_df = grouped_df.agg(countDistinct("visit_date").alias("ct_days_with_outpatient_visit"),
                           pyspark_sum("outpatient_visit_count").alias("sum_outpatient_visit_count"))

result_df = result_df.filter(col("sum_outpatient_visit_count") >= 1)

ct_days_with_outpatient_visit = result_df.drop("sum_outpatient_visit_count")

In [ ]:
ct_days_with_outpatient_visit.show()

+----------+---------------------+-------------------+-----------------------------+
|patient_id|enrollment_start_date|enrollment_end_date|ct_days_with_outpatient_visit|
+----------+---------------------+-------------------+-----------------------------+
|    ID0852|           2023-10-01|         2023-10-31|                            3|
|    ID0014|           2023-05-01|         2023-05-31|                            3|
|    ID0061|           2023-12-01|         2023-12-31|                            3|
|    ID0889|           2023-05-01|         2023-05-31|                            2|
|    ID0070|           2023-01-01|         2023-01-31|                            3|
|    ID0530|           2023-09-01|         2023-09-30|                            1|
|    ID0512|           2023-04-01|         2023-04-30|                            3|
|    ID0480|           2023-05-01|         2023-05-31|                            2|
|    ID0432|           2023-03-01|         2023-03-31|           

### Checks

In [ ]:
# Expect either no records lost OR some records lost

ct_days_with_outpatient_visit.count()

Out[88]: 4532

# Results CSV

In [ ]:
# Need to join the two outpatient results to get the final schema
from pyspark.sql.functions import col

joined_df = ct_outpatient_visits.join(ct_days_with_outpatient_visit,
                                 (ct_days_with_outpatient_visit.patient_id == ct_outpatient_visits.patient_id) &
                                 (ct_days_with_outpatient_visit.enrollment_start_date == ct_outpatient_visits.enrollment_start_date) &
                                 (ct_days_with_outpatient_visit.enrollment_end_date == ct_outpatient_visits.enrollment_end_date),
                                 "left")

result_df = joined_df.select(ct_outpatient_visits.patient_id, ct_outpatient_visits.enrollment_start_date, ct_outpatient_visits.enrollment_end_date, "ct_outpatient_visits", coalesce("ct_days_with_outpatient_visit", lit(0)).alias("ct_days_with_outpatient_visit")
).distinct()

In [ ]:
result_df.show()

+----------+---------------------+-------------------+--------------------+-----------------------------+
|patient_id|enrollment_start_date|enrollment_end_date|ct_outpatient_visits|ct_days_with_outpatient_visit|
+----------+---------------------+-------------------+--------------------+-----------------------------+
|    ID0137|           2023-04-01|         2023-04-30|                 1.0|                            1|
|    ID0471|           2023-02-01|         2023-02-28|                 0.0|                            0|
|    ID0587|           2023-10-01|         2023-10-31|                 0.0|                            0|
|    ID0204|           2023-05-01|         2023-05-31|                 6.0|                            2|
|    ID0516|           2023-10-01|         2023-10-31|                 0.0|                            0|
|    ID0937|           2023-11-01|         2023-11-30|                 3.0|                            1|
|    ID0126|           2023-02-01|         202

In [ ]:
result_df.count()

Out[91]: 7309

### Write & Read Result CSV

In [ ]:
# s3_output_path = "s3://waymark-assignment/result.csv"
# result_df.write.option("header", True).mode("overwrite").csv(s3_output_path)

local_output_path = "/Users/saratometich/Downloads/result.csv"
result_df.write.option("header", True).mode("overwrite").csv(local_output_path)

In [ ]:
result_csv = spark.read.options(header='True').csv("/Users/saratometich/Downloads/result.csv")

## Checks

In [ ]:
result_csv.count()

Out[94]: 7309

In [ ]:
count_check = result_csv.filter(col("ct_days_with_outpatient_visit") > 0).select("patient_id", "enrollment_start_date", "enrollment_end_date").distinct()

count_check.count()

Out[95]: 4532

In [ ]:
count_check_2 = result_csv.filter(col("ct_days_with_outpatient_visit") == 0).select("patient_id", "enrollment_start_date", "enrollment_end_date").distinct()

count_check_2.count()

Out[96]: 2777

## Check example patient ID

In [ ]:
example_check_result = result_csv.filter(col("patient_id") == 'ID0073').select("*").distinct().orderBy("enrollment_start_date")

example_check_result.show()

+----------+---------------------+-------------------+--------------------+-----------------------------+
|patient_id|enrollment_start_date|enrollment_end_date|ct_outpatient_visits|ct_days_with_outpatient_visit|
+----------+---------------------+-------------------+--------------------+-----------------------------+
|    ID0073|           2023-02-01|         2023-02-28|                10.0|                            2|
|    ID0073|           2023-03-01|         2023-03-31|                 5.0|                            2|
|    ID0073|           2023-04-01|         2023-04-30|                 2.0|                            1|
|    ID0073|           2023-05-01|         2023-05-31|                 0.0|                            0|
|    ID0073|           2023-06-01|         2023-06-30|                 3.0|                            1|
|    ID0073|           2023-08-01|         2023-08-31|                 4.0|                            1|
|    ID0073|           2023-09-01|         202

In [ ]:
example_check_span = enrollment_span_csv.filter(col("patient_id") == 'ID0073').select("*").distinct().orderBy("enrollment_start_date")

example_check_span.show()

+----------+---------------------+-------------------+
|patient_id|enrollment_start_date|enrollment_end_date|
+----------+---------------------+-------------------+
|    ID0073|           2023-02-01|         2023-02-28|
|    ID0073|           2023-03-01|         2023-03-31|
|    ID0073|           2023-04-01|         2023-04-30|
|    ID0073|           2023-05-01|         2023-05-31|
|    ID0073|           2023-06-01|         2023-06-30|
|    ID0073|           2023-08-01|         2023-08-31|
|    ID0073|           2023-09-01|         2023-09-30|
|    ID0073|           2023-11-01|         2023-11-30|
+----------+---------------------+-------------------+




# Answer 2

Report the number of distinct values of ct_days_with_outpatient_visit in
result.csv

ANSWER: 9 (including 0 as a distinct value)

In [ ]:
distinct_values = result_csv.select("ct_days_with_outpatient_visit").distinct()

distinct_values.count() # includes 0/null

Out[99]: 9

## Checks

In [ ]:
distinct_values = result_csv.select("ct_days_with_outpatient_visit").distinct().orderBy("ct_days_with_outpatient_visit")

distinct_values.show()

+-----------------------------+
|ct_days_with_outpatient_visit|
+-----------------------------+
|                            0|
|                            1|
|                            2|
|                            3|
|                            4|
|                            5|
|                            6|
|                            7|
|                            8|
+-----------------------------+



In [ ]:
distinct_values = ct_days_with_outpatient_visit.select("ct_days_with_outpatient_visit").distinct().orderBy("ct_days_with_outpatient_visit")

distinct_values.show()

+-----------------------------+
|ct_days_with_outpatient_visit|
+-----------------------------+
|                            1|
|                            2|
|                            3|
|                            4|
|                            5|
|                            6|
|                            7|
|                            8|
+-----------------------------+



# QA Section

I used SQL to check my results

## Enrollment Span Content Checks

In [ ]:
enrollment_span_csv.createOrReplaceTempView("enrollment_span")

### Enrollment Date Reasonability Checks

In [ ]:
spark.sql("""

  select distinct
  patient_id
  , enrollment_start_date
  , enrollment_end_date

  from enrollment_span

  where enrollment_start_date < enrollment_end_date

"""
).count()

Out[103]: 7309

In [ ]:
spark.sql("""

  select distinct
  patient_id
  , enrollment_start_date
  , enrollment_end_date

  from enrollment_span

  where enrollment_start_date = enrollment_end_date

"""
).count()

Out[104]: 0

In [ ]:
spark.sql("""

  select distinct
  patient_id
  , enrollment_start_date
  , enrollment_end_date

  from enrollment_span

  where enrollment_start_date > enrollment_end_date

"""
).count()

Out[105]: 0

### Duplicates

- Make sure no duplicate rows exist

In [ ]:
spark.sql("""

  select distinct
  patient_id
  , enrollment_start_date
  , enrollment_end_date

  from enrollment_span

  group by
  patient_id
  , enrollment_start_date
  , enrollment_end_date

  having count(*) > 1

"""
).count()

Out[106]: 0

### Min/Max Enrollment Dates

- Check max/min enrollment dates to come up the reasonability checks

In [ ]:
spark.sql("""

    select distinct
    min(enrollment_start_date)
    , max(enrollment_start_date)
    , min(enrollment_end_date)
    , max(enrollment_end_date)

    from enrollment_span

"""
).show()

+--------------------------+--------------------------+------------------------+------------------------+
|min(enrollment_start_date)|max(enrollment_start_date)|min(enrollment_end_date)|max(enrollment_end_date)|
+--------------------------+--------------------------+------------------------+------------------------+
|                2023-01-01|                2023-12-01|              2023-01-31|              2023-12-31|
+--------------------------+--------------------------+------------------------+------------------------+



### Reasonability Check on Enrollment Record Count

- Now that we know min and max, make sure members are not enrolled for more than 12 months

In [ ]:
spark.sql("""

    select distinct
    patient_id

    from enrollment_span

    group by
    patient_id

    having count(distinct enrollment_start_date) > 12

"""
).count()

Out[108]: 0

### Check for 12-Month Continuous Enrollment

- Any patients with continuous enrollment for all 12 months?

In [ ]:
spark.sql("""

    select distinct
    patient_id

    from enrollment_span

    group by
    patient_id

    having count(distinct enrollment_start_date) = 12

"""
).count()

Out[109]: 33

## Outpatient Visits Content Checks

In [ ]:
op_visits_clean.createOrReplaceTempView("op_visits_csv")

In [ ]:
ct_outpatient_visits.createOrReplaceTempView("outpatient_visits")

#### Get an example patient ID

In [ ]:
spark.sql("""

    select *

    from enrollment_span

    where patient_id = 'ID0001'

"""
).show()

+----------+---------------------+-------------------+
|patient_id|enrollment_start_date|enrollment_end_date|
+----------+---------------------+-------------------+
|    ID0001|           2023-01-01|         2023-01-31|
|    ID0001|           2023-02-01|         2023-02-28|
|    ID0001|           2023-06-01|         2023-06-30|
|    ID0001|           2023-11-01|         2023-11-30|
|    ID0001|           2023-12-01|         2023-12-31|
|    ID0001|           2023-03-01|         2023-03-31|
|    ID0001|           2023-08-01|         2023-08-31|
+----------+---------------------+-------------------+



In [ ]:
spark.sql("""

    select
    *
    , sum(outpatient_visit_count) over (partition by patient_id, date_trunc('MONTH', visit_date)) as ct_outpatient_visits

    from op_visits_csv

    where patient_id = 'ID0001'

"""
).show()

+----------+----------------------+----------+--------------------+
|patient_id|outpatient_visit_count|visit_date|ct_outpatient_visits|
+----------+----------------------+----------+--------------------+
|    ID0001|                     4|2023-01-21|                13.0|
|    ID0001|                     1|2023-01-25|                13.0|
|    ID0001|                     2|2023-01-03|                13.0|
|    ID0001|                     2|2023-01-15|                13.0|
|    ID0001|                     2|2023-01-08|                13.0|
|    ID0001|                     2|2023-01-09|                13.0|
|    ID0001|                     1|2023-03-22|                 4.0|
|    ID0001|                     3|2023-03-11|                 4.0|
|    ID0001|                     3|2023-06-14|                 6.0|
|    ID0001|                     2|2023-06-03|                 6.0|
|    ID0001|                     1|2023-06-03|                 6.0|
|    ID0001|                     4|2023-07-17|  

#### Check ct_outpatient_visits

- Compare SQL output to python output

In [ ]:
spark.sql("""

    select distinct
    enr.patient_id
    , enrollment_start_date
    , enrollment_end_date
    , coalesce(sum(outpatient_visit_count), 0) as ct_outpatient_visits

    from enrollment_span enr

    left join op_visits_csv op
        on op.patient_id = enr.patient_id
        and op.visit_date <= enr.enrollment_end_date
        and op.visit_date >= enr.enrollment_start_date

    group by
    enr.patient_id
    , enrollment_start_date
    , enrollment_end_date

"""
).show()

+----------+---------------------+-------------------+--------------------+
|patient_id|enrollment_start_date|enrollment_end_date|ct_outpatient_visits|
+----------+---------------------+-------------------+--------------------+
|    ID0741|           2023-08-01|         2023-08-31|                 4.0|
|    ID0437|           2023-07-01|         2023-07-31|                 5.0|
|    ID0734|           2023-04-01|         2023-04-30|                 5.0|
|    ID0797|           2023-10-01|         2023-10-31|                 2.0|
|    ID0864|           2023-02-01|         2023-02-28|                 2.0|
|    ID0088|           2023-07-01|         2023-07-31|                18.0|
|    ID0852|           2023-10-01|         2023-10-31|                 5.0|
|    ID0211|           2023-11-01|         2023-11-30|                 0.0|
|    ID0084|           2023-01-01|         2023-01-31|                 9.0|
|    ID0530|           2023-09-01|         2023-09-30|                 3.0|
|    ID0229|

In [ ]:
spark.sql("""

    select *

    from outpatient_visits

    where patient_id = 'ID0741'
    and enrollment_start_date = '2023-08-01'

"""
).show()

+---------------------+-------------------+----------+--------------------+
|enrollment_start_date|enrollment_end_date|patient_id|ct_outpatient_visits|
+---------------------+-------------------+----------+--------------------+
|           2023-08-01|         2023-08-31|    ID0741|                 4.0|
+---------------------+-------------------+----------+--------------------+



## Days with Outpatient Visits Content Checks

In [ ]:
ct_days_with_outpatient_visit.createOrReplaceTempView("outpatient_days")

### Check ct_days_with_outpatient_visit

- Check output using SQL

In [ ]:
spark.sql("""

    select distinct
    enr.patient_id
    , enrollment_start_date
    , enrollment_end_date
    , count(distinct visit_date) as ct_days_with_outpatient_visit

    from enrollment_span enr

    left join op_visits_csv op
        on op.patient_id = enr.patient_id
        and op.visit_date <= enr.enrollment_end_date
        and op.visit_date >= enr.enrollment_start_date

    group by
    enr.patient_id
    , enrollment_start_date
    , enrollment_end_date

    having sum(outpatient_visit_count) >= 1

"""
).show()

+----------+---------------------+-------------------+-----------------------------+
|patient_id|enrollment_start_date|enrollment_end_date|ct_days_with_outpatient_visit|
+----------+---------------------+-------------------+-----------------------------+
|    ID0852|           2023-10-01|         2023-10-31|                            3|
|    ID0014|           2023-05-01|         2023-05-31|                            3|
|    ID0061|           2023-12-01|         2023-12-31|                            3|
|    ID0889|           2023-05-01|         2023-05-31|                            2|
|    ID0070|           2023-01-01|         2023-01-31|                            3|
|    ID0530|           2023-09-01|         2023-09-30|                            1|
|    ID0512|           2023-04-01|         2023-04-30|                            3|
|    ID0480|           2023-05-01|         2023-05-31|                            2|
|    ID0432|           2023-03-01|         2023-03-31|           

In [ ]:
spark.sql("""

    select *

    from outpatient_days

    where patient_id = 'ID0852'
    and enrollment_start_date = '2023-10-01'

"""
).show()

+----------+---------------------+-------------------+-----------------------------+
|patient_id|enrollment_start_date|enrollment_end_date|ct_days_with_outpatient_visit|
+----------+---------------------+-------------------+-----------------------------+
|    ID0852|           2023-10-01|         2023-10-31|                            3|
+----------+---------------------+-------------------+-----------------------------+



In [ ]:
spark.sql("""

    select *

    from op_visits_csv

    where patient_id = 'ID0852'
    and date_trunc('MONTH', visit_date) = '2023-10-01'

"""
).show()

+----------+----------------------+----------+
|patient_id|outpatient_visit_count|visit_date|
+----------+----------------------+----------+
|    ID0852|                     2|2023-10-15|
|    ID0852|                     1|2023-10-07|
|    ID0852|                     2|2023-10-24|
+----------+----------------------+----------+



### Verify patient IDs that didn't meet the criteria

- The number of distinct days within an enrollment period (between enrollment_start_date and enrollment_end_date) when the patient had one or more outpatient visit

In [ ]:
spark.sql("""

select distinct patient_id from outpatient_visits
except
select distinct patient_id from outpatient_days

"""
).show()

+----------+
|patient_id|
+----------+
|    ID0639|
|    ID0795|
|    ID0981|
|    ID0627|
|    ID0207|
|    ID0815|
|    ID0783|
|    ID0507|
|    ID0971|
|    ID0963|
|    ID0107|
|    ID0079|
|    ID0427|
|    ID0979|
|    ID0911|
|    ID0691|
|    ID0879|
|    ID0607|
|    ID0759|
|    ID0203|
+----------+
only showing top 20 rows



In [ ]:
spark.sql("""

    select sum(ct_outpatient_visits)

    from outpatient_visits

    where patient_id in (

        select distinct patient_id from outpatient_visits
        except
        select distinct patient_id from outpatient_days

    )

"""
).show()

+-------------------------+
|sum(ct_outpatient_visits)|
+-------------------------+
|                      0.0|
+-------------------------+



In [ ]:
spark.sql("""

    select *

    from op_visits_csv

    where patient_id in (

        select distinct patient_id from outpatient_visits
        except
        select distinct patient_id from outpatient_days

    )

"""
).show()

+----------+----------------------+----------+
|patient_id|outpatient_visit_count|visit_date|
+----------+----------------------+----------+
+----------+----------------------+----------+



In [ ]:
spark.sql("""

    select *

    from outpatient_visits

    where patient_id = 'ID0647'

"""
).show()

+---------------------+-------------------+----------+--------------------+
|enrollment_start_date|enrollment_end_date|patient_id|ct_outpatient_visits|
+---------------------+-------------------+----------+--------------------+
|           2023-06-01|         2023-06-30|    ID0647|                 0.0|
|           2023-03-01|         2023-03-31|    ID0647|                 0.0|
|           2023-01-01|         2023-01-31|    ID0647|                 0.0|
|           2023-02-01|         2023-02-28|    ID0647|                 0.0|
|           2023-11-01|         2023-11-30|    ID0647|                 0.0|
|           2023-09-01|         2023-09-30|    ID0647|                 0.0|
+---------------------+-------------------+----------+--------------------+



In [ ]:
spark.sql("""

    select *

    from op_visits_csv

    where patient_id = 'ID0647'

"""
).show()

+----------+----------------------+----------+
|patient_id|outpatient_visit_count|visit_date|
+----------+----------------------+----------+
+----------+----------------------+----------+

